In [1]:
import toytree
import ipcoal
import numpy as np
import pandas as pd
def get_num_edges_at_time(tree, time):
    nodes_above = ([idx for idx, node in tree.idx_dict.items() if node.height > time])
    edges_above = len(nodes_above) + 1
    return edges_above
def get_tree_clade_times(tree):
    nodes_ = []
    heights_ = []
    for curr_node in tree.treenode.traverse():
        if not curr_node.is_leaf():
            nodes_.append(curr_node.get_leaf_names())
            heights_.append(curr_node.height)
    pddf = pd.DataFrame([nodes_,heights_],index=['clades','heights']).T
    return(pddf)

In [2]:
def get_tree_clade_times(tree):
    nodes_ = []
    heights_ = []
    for curr_node in tree.treenode.traverse():
        if not curr_node.is_leaf():
            nodes_.append(curr_node.get_leaf_names())
            heights_.append(curr_node.height)
    pddf = pd.DataFrame([nodes_,heights_],index=['clades','heights']).T
    return(pddf)

In [45]:
[1,2,3,4,5][-2]

4

In [49]:
def get_branch_intervals(tr, gt, br):
    '''
    tr = species tree with Ne attribute
    gt = gene tree simulated on that species tree
    br = treenode representing a branch on the tree
    '''
    st_times = get_tree_clade_times(tr)
    gt_times = get_tree_clade_times(gt)
    coalclade = br.get_leaf_names()
    
    ###temp
    st_coal_node = tr.treenode.search_nodes(idx=tr.get_mrca_idx_from_tip_labels(br.get_leaf_names()))[0]
    nearest_st_node = st_coal_node
    while ((nearest_st_node.height + nearest_st_node.dist) < br.height):
        nearest_st_node = nearest_st_node.up
    coalclade = nearest_st_node.get_leaf_names()
    ###
    
    
    br_lower = br.height
    br_upper = br_lower + br.dist
    gt_clade_changes = (gt_times.heights < br_upper) & (gt_times.heights > br_lower)
    st_clade_changes = (st_times.heights < br_upper) & (st_times.heights > br_lower)
    st_time_diffed = st_times[st_clade_changes]
    #return(np.array([all(elem in clade for elem in coalclade) for clade in st_time_diffed.clades]))

    contains_clade = st_time_diffed[np.array([all(elem in clade for elem in coalclade) for clade in st_time_diffed.clades])]

    if not len(contains_clade.columns):
        contains_clade = pd.DataFrame(columns=['clades','heights'])
    contains_clade = pd.DataFrame([list(contains_clade.clades.append(pd.Series([coalclade]),ignore_index=True)),list(contains_clade.heights.append(pd.Series(br_lower)))],index=['clades','heights']).T
    contains_clade = contains_clade.sort_values('heights')

    all_members = []
    for i in contains_clade.clades:
        all_members.extend(i)
    all_members = np.unique(all_members)
    
    relevant_coals = pd.DataFrame(columns=["heights"])

    if np.sum(gt_clade_changes):
        potential_coals = gt_times[gt_clade_changes]
        relevant_coals = potential_coals[[set(i).issubset(all_members) for i in potential_coals.clades]]
        relevant_coals = relevant_coals.sort_values('heights')

    time_points = np.sort(list(contains_clade.heights) + list(relevant_coals.heights) + [br_upper])
    if int(time_points[-1]) == int(time_points[-2]):
        time_points = time_points[:-1]
    starts = time_points[:-1]
    stops = time_points[1:]
    lengths = stops-starts
    num_to_coal = np.repeat(1,len(starts))
    ne = np.repeat(1,len(starts))
    a_df = pd.DataFrame([starts,stops,lengths,num_to_coal,ne],index=['starts','stops','lengths','num_to_coal','ne']).T
    mids = (a_df.stops + a_df.starts)/2
    interval_reduced_trees=[]
    
    nes = []
    for mid in mids:
        clade = contains_clade.clades.iloc[np.sum(contains_clade.heights<mid)-1]

        cladeNe = tr.treenode.search_nodes(idx=tr.get_mrca_idx_from_tip_labels(clade))[0].Ne
        nes.append(cladeNe)
        reduced_tree = gt.prune(clade)
        interval_reduced_trees.append(reduced_tree.newick)
        
    a_df['reduced_trees'] = interval_reduced_trees
    a_df['mids'] = mids
    a_df['ne'] = nes
    a_df['num_to_coal'] = a_df.apply(lambda x: get_num_edges_at_time(toytree.tree(x['reduced_trees']), x['mids']), axis=1)
    
    return a_df

In [5]:
tre = toytree.rtree.bdtree(12,time=1e6,seed=123)

In [6]:
tre = tre.mod.node_scale_root_height(treeheight=1e6)

In [7]:
# set a random Ne to each node
node_ne_dict = {i:np.random.randint(100,100000) for i in range(tre.nnodes)}
tre = tre.set_node_data('Ne',node_ne_dict)

In [8]:
tre.draw(ts='p',node_labels=True,node_sizes=15,width=500,height=500,node_mask=False);

<svg class="toyplot-canvas-Canvas" xmlns:toyplot="http://www.sandia.gov/toyplot" xmlns:xlink="http://www.w3.org/1999/xlink" xmlns="http://www.w3.org/2000/svg" width="500.0px" height="500.0px" viewBox="0 0 500.0 500.0" preserveAspectRatio="xMidYMid meet" style="background-color:transparent;border-color:#292724;border-style:none;border-width:1.0;fill:rgb(16.1%,15.3%,14.1%);fill-opacity:1.0;font-family:Helvetica;font-size:12px;opacity:1.0;stroke:rgb(16.1%,15.3%,14.1%);stroke-opacity:1.0;stroke-width:1.0" id="taeade4ef4ade49e7a838de76aad63141"> 0 1 2 3 4 5 6 7 8 9 10 11 12 13 14 15 16 17 18 19 20 21 22 r0 r1 r2 r3 r4 r5 r6 r7 r8 r9 r10 r11 0 250000 500000 750000 1000000

In [9]:
tre.treenode.search_nodes(idx=22)[0].Ne

27662

In [10]:
tre.treenode.height

1000000.0

In [11]:
mod = ipcoal.Model(tre,Ne=350000,seed_trees=1235)
mod.sim_trees(1)

In [12]:
gtr = toytree.tree(mod.df.genealogy[0])
gtr.draw(ts='p',node_labels=True,node_sizes=15,width=500,height=500,node_mask=False);

<svg class="toyplot-canvas-Canvas" xmlns:toyplot="http://www.sandia.gov/toyplot" xmlns:xlink="http://www.w3.org/1999/xlink" xmlns="http://www.w3.org/2000/svg" width="500.0px" height="500.0px" viewBox="0 0 500.0 500.0" preserveAspectRatio="xMidYMid meet" style="background-color:transparent;border-color:#292724;border-style:none;border-width:1.0;fill:rgb(16.1%,15.3%,14.1%);fill-opacity:1.0;font-family:Helvetica;font-size:12px;opacity:1.0;stroke:rgb(16.1%,15.3%,14.1%);stroke-opacity:1.0;stroke-width:1.0" id="t48a1ad0e2e134d08a9b309281996a6e8"> 0 1 2 3 4 5 6 7 8 9 10 11 12 13 14 15 16 17 18 19 20 21 22 r11 r8 r9 r5 r4 r6 r3 r7 r10 r2 r1 r0 0 200000 400000 600000 800000 1000000 1200000 1400000

In [13]:
mybranch = gtr.treenode.search_nodes(idx=21)[0]
print(mybranch)


   /-r0
--|
  |   /-r1
   \-|
      \-r2


In [59]:
for blah in [8]:
    mybranch = gtr.treenode.search_nodes(idx=blah)[0]
    df = get_branch_intervals(tre,gtr,mybranch)
df

,starts,stops,lengths,num_to_coal,ne,reduced_trees,mids
0,2.328306e-10,1.200976e+05,120097.615621,1,24711,r10:1.43942e+06;,6.004881e+04
1,1.200976e+05,1.795015e+05,59403.925784,1,24711,r10:1.43942e+06;,1.497996e+05
2,1.795015e+05,2.524446e+05,72943.092497,1,24711,r10:1.43942e+06;,2.159731e+05
3,2.524446e+05,3.105940e+05,58149.408915,1,24711,r10:1.43942e+06;,2.815193e+05
4,3.105940e+05,3.359400e+05,25345.917039,1,24711,r10:1.43942e+06;,3.232670e+05
5,3.359400e+05,3.525514e+05,16611.463474,3,95573,"(r10:1.31429e+06,(r9:903...",3.442457e+05
6,3.525514e+05,5.113543e+05,158802.888453,4,57236,"(r10:1.31429e+06,(r9:903...",4.319529e+05
7,5.113543e+05,7.629566e+05,251602.303500,3,57236,"(r10:1.31429e+06,(r9:903...",6.371555e+05
8,7.629566e+05,9.030085e+05,140051.891918,3,57236,"(r10:1.31429e+06,(r9:903...",8.329826e+05
9,9.030085e+05,1.000000e+06,96991.492799,2,57236,"(r10:1.31429e+06,(r9:903...",9.515043e+05


In [63]:
df.ne

<bound method _flex_comp_method_FRAME.<locals>.f of           starts         stops        lengths  num_to_coal     ne  \
0   2.328306e-10  1.200976e+05  120097.615621            1  24711   
1   1.200976e+05  1.795015e+05   59403.925784            1  24711   
2   1.795015e+05  2.524446e+05   72943.092497            1  24711   
3   2.524446e+05  3.105940e+05   58149.408915            1  24711   
4   3.105940e+05  3.359400e+05   25345.917039            1  24711   
5   3.359400e+05  3.525514e+05   16611.463474            3  95573   
6   3.525514e+05  5.113543e+05  158802.888453            4  57236   
7   5.113543e+05  7.629566e+05  251602.303500            3  57236   
8   7.629566e+05  9.030085e+05  140051.891918            3  57236   
9   9.030085e+05  1.000000e+06   96991.492799            2  57236   
10  1.000000e+06  1.154319e+06  154318.777731            5  27662   
11  1.154319e+06  1.161669e+06    7350.381479            4  27662   
12  1.161669e+06  1.314291e+06  152621.463515      

In [ ]:
def condense_df(df):
    df

In [58]:
df

,starts,stops,lengths,num_to_coal,ne,reduced_trees,mids
0,2.328306e-10,85223.358806,85223.358806,1,35151,r2:1.43942e+06;,42611.679403
1,8.522336e+04,252444.633902,167221.275096,3,28214,"(r0:310594,(r1:252445,r2...",168833.996354


In [35]:
df

(0    1.15432e+06
 Name: heights, dtype: object,
 4    1.16167e+06
 Name: heights, dtype: object,
 1161669.1592098707)

In [120]:
test = pd.DataFrame(columns=['clades','heights'])#[np.array([])]

In [121]:
if not len(test.columns):
    print('yep')

yep


In [107]:
df

array([], dtype=float64)

In [89]:
df

,starts,stops,lengths,num_to_coal,ne,reduced_trees,mids
0,4.656613e-10,1322.289330,1322.289330,1,26782,r5:1.43942e+06;,661.144665
1,1.322289e+03,93078.787928,91756.498598,2,82605,"(r4:120098,r5:120098);",47200.538629
2,9.307879e+04,120097.615621,27018.827693,3,22986,"(r6:179502,(r4:120098,r5...",106588.201775


In [60]:
df

,clades,heights
2,[r5],4.65661e-10
1,"[r5, r4]",1322.29
0,"[r6, r5, r4]",93078.8


In [57]:
df

""


In [48]:
df = get_branch_intervals(tre,gtr,mybranch)
df

                        clades heights
0  [r11, r10, r9, r8, r7, r...   1e+06
1           [r11, r10, r9, r8]  352551
2  [r7, r6, r5, r4, r3, r2,...  743481
3                [r10, r9, r8]  335940
4             [r7, r6, r5, r4]  689382


,starts,stops,lengths,num_to_coal,ne,reduced_trees,mids
0,3.105940e+05,5.113543e+05,200760.268966,2,33898,"((r0:310594,(r1:252445,r...",4.109742e+05
1,5.113543e+05,7.434806e+05,232126.246340,2,33898,"((r0:310594,(r1:252445,r...",6.274174e+05
2,7.434806e+05,7.629566e+05,19476.057159,4,19268,"((r0:310594,(r1:252445,r...",7.532186e+05
3,7.629566e+05,9.030085e+05,140051.891918,3,19268,"((r0:310594,(r1:252445,r...",8.329826e+05
4,9.030085e+05,1.000000e+06,96991.492799,3,19268,"((r0:310594,(r1:252445,r...",9.515043e+05
5,1.000000e+06,1.154319e+06,154318.777731,5,36863,"((r0:310594,(r1:252445,r...",1.077159e+06
6,1.154319e+06,1.161669e+06,7350.381479,4,36863,"((r0:310594,(r1:252445,r...",1.157994e+06
7,1.161669e+06,1.314291e+06,152621.463515,3,36863,"((r0:310594,(r1:252445,r...",1.237980e+06
8,1.314291e+06,1.439421e+06,125130.633044,2,36863,"((r0:310594,(r1:252445,r...",1.376856e+06


In [25]:
df = get_branch_intervals(tre,gtr,mybranch)
df

                        clades heights
0  [r11, r10, r9, r8, r7, r...   1e+06
1           [r11, r10, r9, r8]  352551
2  [r7, r6, r5, r4, r3, r2,...  743481
3                [r10, r9, r8]  335940
4             [r7, r6, r5, r4]  689382


,starts,stops,lengths,num_to_coal,ne,reduced_trees,mids
0,3.105940e+05,5.113543e+05,200760.268966,1,15612,"(r0:310594,(r1:252445,r2...",4.109742e+05
1,5.113543e+05,7.434806e+05,232126.246340,1,15612,"(r0:310594,(r1:252445,r2...",6.274174e+05
2,7.434806e+05,7.629566e+05,19476.057159,4,19268,"((r0:310594,(r1:252445,r...",7.532186e+05
3,7.629566e+05,9.030085e+05,140051.891918,3,19268,"((r0:310594,(r1:252445,r...",8.329826e+05
4,9.030085e+05,1.000000e+06,96991.492799,3,19268,"((r0:310594,(r1:252445,r...",9.515043e+05
5,1.000000e+06,1.154319e+06,154318.777731,5,36863,"((r0:310594,(r1:252445,r...",1.077159e+06
6,1.154319e+06,1.161669e+06,7350.381479,4,36863,"((r0:310594,(r1:252445,r...",1.157994e+06
7,1.161669e+06,1.314291e+06,152621.463515,3,36863,"((r0:310594,(r1:252445,r...",1.237980e+06
8,1.314291e+06,1.439421e+06,125130.633044,2,36863,"((r0:310594,(r1:252445,r...",1.376856e+06


In [21]:
df.reduced_trees[0]

'(r0:310594,(r1:252445,r2:252445)0:58149.4);'

In [3]:
t=toytree.rtree.rtree(5)

In [4]:
t.draw();

<svg class="toyplot-canvas-Canvas" xmlns:toyplot="http://www.sandia.gov/toyplot" xmlns:xlink="http://www.w3.org/1999/xlink" xmlns="http://www.w3.org/2000/svg" width="260.0px" height="275.0px" viewBox="0 0 260.0 275.0" preserveAspectRatio="xMidYMid meet" style="background-color:transparent;border-color:#292724;border-style:none;border-width:1.0;fill:rgb(16.1%,15.3%,14.1%);fill-opacity:1.0;font-family:Helvetica;font-size:12px;opacity:1.0;stroke:rgb(16.1%,15.3%,14.1%);stroke-opacity:1.0;stroke-width:1.0" id="t292dc781eae844e2814704ad1acdd001"> r0 r1 r2 r3 r4

In [7]:
t.prune(['r1','r0','r2']).draw();

<svg class="toyplot-canvas-Canvas" xmlns:toyplot="http://www.sandia.gov/toyplot" xmlns:xlink="http://www.w3.org/1999/xlink" xmlns="http://www.w3.org/2000/svg" width="260.0px" height="275.0px" viewBox="0 0 260.0 275.0" preserveAspectRatio="xMidYMid meet" style="background-color:transparent;border-color:#292724;border-style:none;border-width:1.0;fill:rgb(16.1%,15.3%,14.1%);fill-opacity:1.0;font-family:Helvetica;font-size:12px;opacity:1.0;stroke:rgb(16.1%,15.3%,14.1%);stroke-opacity:1.0;stroke-width:1.0" id="td6a815a6ff8a49f5983206ad866e427f"> r2 r0 r1

In [8]:
t.prune(['r0','r1','r2','r3','r4']).draw();

<svg class="toyplot-canvas-Canvas" xmlns:toyplot="http://www.sandia.gov/toyplot" xmlns:xlink="http://www.w3.org/1999/xlink" xmlns="http://www.w3.org/2000/svg" width="260.0px" height="275.0px" viewBox="0 0 260.0 275.0" preserveAspectRatio="xMidYMid meet" style="background-color:transparent;border-color:#292724;border-style:none;border-width:1.0;fill:rgb(16.1%,15.3%,14.1%);fill-opacity:1.0;font-family:Helvetica;font-size:12px;opacity:1.0;stroke:rgb(16.1%,15.3%,14.1%);stroke-opacity:1.0;stroke-width:1.0" id="td27ead8ec9084cf1a878d9aaeb353f90"> r0 r1 r2 r3 r4